In [1]:
%load_ext autoreload
%autoreload 2

import os
import zarr
import dask.array as da
import napari
import sys
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy.orm import Session

from ete3 import Tree

sys.path.append('../tracks_interactions/tracks_interactions')
from db.db_model import Base, CellDB, TrackDB
import graph.family_graph as fg

In [2]:
os.environ['NAPARI_PERFMON'] = '0'

In [3]:
labels_zarr_path = r'D:\kasia\tracking\E6_exp\code\tests\example_track_labels.zarr'
labels = da.from_zarr(labels_zarr_path)

In [4]:
# display the labels
viewer = napari.Viewer()
labels_layer = viewer.add_labels(labels,name='Labels')
fg.viewer = viewer

c:\Users\kmk280\.conda\envs\tracking\lib\site-packages\napari\_qt\qt_main_window.py:543: UserWarning: Using NAPARI_PERFMON with an already-running QtApp (--gui qt?) is not supported.
  qapp = get_app()


In [5]:
# get access to the database
new_db_path = r'D:\kasia\tracking\E6_exp\code\gardener_20_dev\cells_database_2tables.db'
engine = create_engine(f'sqlite:///{new_db_path}')

# send engine to the library
fg.engine = engine

In [6]:
# create a plot widget
t_max = viewer.dims.range[0][1]
plot_widget = fg.build_lineage_widget(t_max)
fg.plot_widget = plot_widget


# add lineage graph
viewer.window.add_dock_widget(plot_widget,area='bottom',name='family')

In [7]:
# connect lineage graph update
labels_layer.events.selected_label.connect(fg.update_lineage_display)

<function graph.family_graph.update_lineage_display(event)>

In [8]:
from db.db_functions import get_descendants

ModuleNotFoundError: No module named 'tracks_interactions'

### Test

In [9]:
my_root = 1

plot_view = plot_widget.getItem(0,0)
plot_view.clear()

# buid the tree
tree = fg.build_Newick_tree(engine, my_root)

In [10]:
# update the widget with the tree
plot_view = fg.render_tree_view(plot_view,tree,viewer)